In [10]:
# !pip install praw prawcore nltk vaderSentiment bertopic

In [8]:
import praw
import torch

if torch.cuda.is_available():
    device = torch.device("cuda:0")  # Use the first GPU
    print("using GPU")
else:
    device = torch.device("cpu")  # Use CPU if no GPU is available
    print("using CPU")
    

using GPU


In [4]:
# @title
highest_karma_string = """1. CheetahSperm18
46,727,608

2. GallowBoob
36,906,592

3. regian24
32,809,063

4. mvea
29,018,732

5. My_Memes_Will_Cure_U
26,931,090

6. TooShiftyForYou
26,243,536

7. pepsi_next
24,122,213

8. Master1718
23,984,371

9. Lol33ta
22,094,521

10. BunyipPouch
18,504,484

11. KevlarYarmulke
17,609,843

12. kevinowdziej
17,069,202

13. DaFunkJunkie
16,730,545

14. icant-chooseone
16,056,571

15. SrGrafo
15,702,206

16. SlimJones123
15,548,854

17. memezzer
15,265,560

18. maxwellhill
15,062,329

19. ExpertAccident
14,785,370

20. Sariel007
14,660,885

21. ManiaforBeatles
14,606,505

22. BelleAriel
14,365,557

23. aloofloofah
13,662,662

24. to_the_tenth_power
12,876,777

25. 5_Frog_Margin
12,796,234

26. Holofan4life
12,591,345

27. unnaturalorder
12,539,298

28. iBleeedorange
12,495,730

29. Hummerous
12,293,502

30. commonvanilla
12,267,282

31. speckz
11,951,732

32. Scaulbylausis
11,893,276

33. Turbostrider27
11,559,961

34. asilvertintedrose
11,472,300

35. lrlOurPresident
11,362,476

36. Sumit316
11,284,634

37. imagepoem
11,200,263

38. johnlen1n
11,024,106

39. Wagamaga
11,019,880

40. suzukigun4life
10,738,314

41. mepper
10,736,533

42. MyNameGifOreilly
10,647,958

43. iam4real
10,371,132

44. AristonD
10,351,314

45. Thund3rbolt
10,286,864

46. lnfinity
10,128,822

47. MarvelsGrantMan136
9,950,102

48. esberat
9,895,721

49. DL2828
9,606,281

50. Frocharocha
9,598,824

51. chanma50
9,354,300

52. bubblr
9,285,479

53. 1Voice1Life
9,229,566

54. mike_pants
9,199,686

55. Thedepressionoftrees
9,148,497

56. hildebrand_rarity
8,909,669

57. HellsJuggernaut
8,907,499

58. relevantlife
8,875,254

59. Poem_for_your_sprog
8,820,169

60. dickfromaccounting
8,811,588

61. -eDgAR-
8,780,849

62. poopellar
8,758,336

63. Pirate_Redbeard
8,680,192

64. cyan1618
8,639,461

65. sequence_string
8,519,216

66. Johnnadawearsglasses
8,507,000

67. Boojibs
8,225,274

68. Aztery
8,126,820

69. deathakissaway
8,039,977

70. One_Giant_Nostril
7,873,429

71. DoremusJessup
7,802,486

72. Kuro-Oji
7,756,654

73. jasontaken
7,737,989

74. chrisdh79
7,616,012

75. zsreport
7,550,316

76. BlankVerse
7,414,414

77. RespectMyAuthoriteh
7,404,914

78. pouli-
7,397,989

79. ProgramSupervisor
7,369,702

80. MGLLN
7,362,294

81. St0pX
7,286,902

82. viva_la_vinyl
7,141,898

83. mtlgrems
7,090,101

84. Natsu_1000
7,087,135

85. elee0228
6,935,408

86. Meunderwears
6,878,590

87. sezar4321
6,852,284

88. MiniLoona
6,846,953

89. killHACKS
6,825,354

90. HornyHeracross
6,822,064

91. onlysame1
6,790,849

92. beerbellybegone
6,740,018

93. AutoLovepon
6,734,967

94. ohnoh18
6,711,259

95. chocolat_ice_cream
6,676,542

96. madazzahatter
6,583,446

97. creedroyce
6,514,803

98. The-Autarkh
6,496,786

99. OwO_Bot
6,481,534

100. Thryloz
6,463,672

"""

In [9]:
users = []
temp_user_ids = highest_karma_string.split(". ")
for user_id in temp_user_ids[1:]:
  user = user_id.split("\n")[0]
  users.append(user)

In [12]:
# users

In [13]:
import praw
import prawcore
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
from datetime import datetime

# from transformers import pipeline
nltk.download('stopwords')

import torch
import numpy as np
import pickle
from bertopic import BERTopic
from wordcloud import WordCloud

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/msaxena4/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
# Reddit API credentials
my_client_id = "C-jEYYIDYXzGxFRq5nDy3w"
my_client_secret = "4TVajUEVHnLPQvQZxAMr8SJBwU2Ckw"
my_user_agent = "spark1217"

# Create a Reddit instance
reddit = praw.Reddit(
    client_id=my_client_id,
    client_secret=my_client_secret,
    user_agent=my_user_agent
)


In [15]:
# function to check if a user exists

def check_if_user_exists(name):
    try:
        reddit.redditor(name).id
    except prawcore.exceptions.NotFound:
        return False
    except AttributeError:
        return False
    return True

In [16]:
# ! pip install tqdm
from tqdm import tqdm

In [ ]:
# getting submission ids for each post for each user

user_errors = []
submission_ids = []

for user in tqdm(users, desc="Processing users", unit="user"):
    if check_if_user_exists(user):
        for comment in reddit.redditor(user).submissions.hot(limit=1000):
            try:
                submission_ids.append(comment.id)
            except:
                user_errors.append(user)
                continue


In [ ]:
# with open('submission_ids.txt', 'w') as f:
#     for id in submission_ids:
#         f.write(f"{id}\n")

In [17]:
# Open the file in read mode
with open('submission_ids.txt', 'r') as f:
    # Read all lines from the file into a list
    submission_ids = f.readlines()

# Now, submission_ids_read is a list containing the lines from the file
# You may want to remove the newline characters from each ID
submission_ids = [id.strip() for id in submission_ids]


In [18]:
len(submission_ids), len(set(submission_ids))

(66922, 66922)

In [21]:
import pandas as pd
from tqdm import tqdm
import time

# Initialize DataFrame
reddit_Udata_dict = {'title': [], 'text':[], 'userId': [], 'createdTime': []}
# pd.DataFrame(columns=['content', 'userId', 'createdTime'])

# Initialize list to store errors
sub_errors = []

# Set the interval to save the data
save_interval = 5000

for idx, post_id in enumerate(tqdm(submission_ids[25000:], desc="Processing submission ids", unit="id")):
    try:
        post = reddit.submission(id=post_id)
        title = post.title
        text = post.selftext
        author = post.author
        createdTime = post.created_utc

        reddit_Udata_dict['title'].append(title)
        reddit_Udata_dict['text'].append(text)
        reddit_Udata_dict['userId'].append(author)
        reddit_Udata_dict['createdTime'].append(createdTime)

        # Save the DataFrame at specified intervals
        if idx % save_interval == 0 and idx > 0:
            # Adjust the filename based on your needs
            reddit_Udata = pd.DataFrame(reddit_Udata_dict) # convert dict to df
            
            filename = f'data/Large_RedditUserdata/{idx}.csv'
            reddit_Udata.to_csv(filename, index=False)

    except Exception as e:
        sub_errors.append((post_id, str(e)))
        continue

# Save the final DataFrame
final_filename = 'data/Large_RedditUserdata/final_Reddit_userdata.csv'
reddit_Udata.to_csv(final_filename, index=False)


Processing submission ids:   2%|▏         | 689/41922 [11:31<11:30:02,  1.00s/id]


KeyboardInterrupt: 

In [ ]:
# reddit_Udata

In [ ]:
# Scraping each post from submission id

reddit_Udata = pd.DataFrame(columns=['content', 'userId','createdTime'])

sub_errors = []

for id in tqdm(submission_ids, desc="Processing submission ids", unit="id"):
    try:
        post = reddit.submission(id=id)
        content = post.title + '\n' + post.selftext

        new_row = {'content': content, 'userId': post.author, 'createdTime': post.created_utc}
        reddit_Udata = reddit_Udata.append(new_row, ignore_index=True)
    except:
        sub_errors.append(id)
        continue


In [9]:
# sub_errors

In [ ]:
# preprocessing functions

# Define a function to remove stop words
def remove_stop_words(text):
  stop_words = nltk.corpus.stopwords.words('english')
  return ' '.join([word for word in text.split() if word not in stop_words])

# Define a function to lowercase the text
def lowercase_text(text):
  return text.lower()

# Remove emojis & punctuation
def replace_emojis(text):
  emoji_pattern = r'[^\w\s]'
  text = str(text)
  return re.sub(emoji_pattern, '', text)

def drop_rows_with_empty_text(df):

    df = df.dropna(subset=['content'])
    df = df[df['content'].str.len() >= 1]
    return df

In [ ]:
# preprocess
reddit_Udata['preprocessed_content'] = reddit_Udata['content'].apply(replace_emojis).apply(remove_stop_words).apply(lowercase_text)
reddit_Udata = drop_rows_with_empty_text(reddit_Udata)
reddit_Udata.drop_duplicates(inplace=True)

In [ ]:
#Define a function to perform sentiment analysis using VADER
def get_sentiment(text):
    sentiment = analyzer.polarity_scores(text)
    return sentiment

In [ ]:
# getting sentiment
reddit_Udata['sentiment'] = reddit_Udata['preprocessed_content'].apply(get_sentiment)